In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# DeepLabCut 2.2+ Toolbox - COLAB
![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1628180434489-T0RIWEJJU0FJVOT6FNVD/maDLC.png?format=800w)

https://github.com/DeepLabCut/DeepLabCut

This notebook illustrates how to, for multi-animal projects, use the cloud-based GPU to:
- create a multi-animal training set
- train a network
- evaluate a network
- analyze novel videos
- assemble animals and tracklets
- create quality check plots!

### This notebook assumes you already have a DLC project folder with labeled data and you uploaded it to your own Google Drive.

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.

This shows the most simple code to do so, but many of the functions have additional features, so please check out the docs on GitHub. We also recommend checking out our preprint, which covers the science of maDLC

**Lauer et al 2021:** https://www.biorxiv.org/content/10.1101/2021.04.30.442096v1




## First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"


In [2]:
#(this will take a few minutes to install all the dependences!)
!pip install "deeplabcut[tf]"
%reload_ext numpy
%reload_ext scipy
%reload_ext matplotlib
%reload_ext mpl_toolkits

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 24.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.3/296.3 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 46.4 M

In [3]:
#a few colab specific things needed:
!pip install --upgrade scikit-image
!pip3 install pickle5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 34.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.19.3
    Uninstalling scikit-image-0.19.3:
      Successfully uninstalled scikit-image-0.19.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=255320 sha256=cdcff51a9ec6dd516cbba3075e0f5da52e0872059f890c7d128005fee4ee2ecd
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5


After the package is installed, please click "restart runtime" if it appears for DLC changes to take effect in your COLAB environment. You will see this button at the output of the cells above ^.

In [4]:
import deeplabcut
import pickle5 as pickle

Loading DLC 2.3.8...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


## Link your Google Drive (with your labeled data):

- This code assumes you locally installed DeepLabCut, created a project, extracted and labeled frames. Be sure to "check Labels" to confirm you are happy with your data. As, these frames are the only thing that is used to train your network. 💪 You can find all the docs to do this here: https://deeplabcut.github.io/DeepLabCut

- Next, place your DLC project folder into you Google Drive- i.e., copy the folder named "Project-YourName-TheDate" into Google Drive.

- Then, click run on the cell below to link this notebook to your Google Drive:

In [ ]:
# #Now, let's link to your Google Drive. Run this cell and follow the authorization instructions:
# #(We recommend putting a copy of the github repo in your google drive if you are using the demo "examples")

# from google.colab import drive
# drive.mount('/content/drive')

## Next, edit the few items below, and click run:


In [51]:
# PLEASE EDIT THIS:
ProjectFolderName = 'judo_uchimata_I_W'
VideoType = 'mp4' #, mp4, MOV, or avi, whatever you uploaded!


# No need to edit this, we are going to assume you put videos you want to analyze in the "videos" folder, but if this is NOT true, edit below:
videofile_path = ['/content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/mmaction_exe_uchimata_I_W_gameover_seianno/'+ProjectFolderName+'/videos/'] #Enter the list of videos or folder to analyze.
print("videofile_path : ", videofile_path)

#No need to edit this, as you set it when you passed the ProjectFolderName (above):
## 学習
path_config_file = '/content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/mmaction_exe_uchimata_I_W_gameover_seianno/'+ProjectFolderName+'/config.yaml'
print("path_config_file : ", path_config_file)
#This creates a path variable that links to your google drive project

videofile_path :  ['/content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/mmaction_exe_uchimata_I_W_gameover_seianno/judo_uchimata_I_W/videos/']
path_config_file :  /content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/mmaction_exe_uchimata_I_W_gameover_seianno/judo_uchimata_I_W/config.yaml


## Create a multi-animal training dataset:

- more info: https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html#create-training-dataset

- please check the text below, edit if needed, and then click run (this can take some time):

In [52]:
#OPTIONAL LEARNING: did you know you can check what each function does by running with a ?
deeplabcut.create_multianimaltraining_dataset?

In [53]:
# ATTENTION:
#which shuffle do you want to create and train?
shuffle = 1 #edit if needed; 1 is the default.

#if you labeled on Windows, please set the windows2linux=True:
deeplabcut.create_multianimaltraining_dataset(path_config_file, Shuffles=[shuffle], net_type="dlcrnet_ms5",windows2linux=False, )

Valid subgraph found...
Utilizing the following graph: [[2, 16], [2, 3], [2, 13], [2, 9], [2, 5], [4, 16], [11, 16], [3, 16], [1, 16], [4, 6], [4, 8], [4, 11], [4, 12], [4, 7], [4, 15], [8, 15], [0, 8], [3, 8], [7, 8], [14, 15], [1, 15], [9, 15], [6, 15], [6, 9], [6, 14], [0, 6], [6, 7], [1, 5], [0, 1], [5, 7], [5, 11], [5, 14], [5, 9], [0, 5], [5, 13], [3, 5], [3, 10], [7, 14], [7, 9], [13, 14], [12, 13], [0, 13], [9, 13], [9, 14], [9, 10], [9, 12], [9, 11], [0, 9], [11, 12], [0, 11], [0, 10]]
Creating training data for: Shuffle: 1 TrainFraction:  0.95


100%|██████████| 64/64 [00:00<00:00, 12382.85it/s]

The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


## Start training:
This function trains the network for a specific shuffle of the training dataset.
 - more info: https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html#train-the-network

In [ ]:
#let's also change the display and save_iters just in case Colab takes away the GPU...
#Typically, you want to train to 50,000 - 200K iterations.
#more info and there are more things you can set: https://github.com/DeepLabCut/DeepLabCut/blob/master/docs/functionDetails.md#g-train-the-network

deeplabcut.train_network(path_config_file, shuffle=shuffle, displayiters=100,saveiters=1000, maxiters=50000, allow_growth=True)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 50K iterations).
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....

**When you hit "STOP" you will get a KeyInterrupt "error"! No worries! :)**

## re-train

In [29]:
try:
    import deeplabcut
    import tkinter
    from tkinter import filedialog

    print(f'Using DeepLabCut version: {deeplabcut. __version__}')

except:
    print("Please run the notebook in in your local environment")

Using DeepLabCut version: 2.3.8


In [ ]:
# model_checkpoint_path: "/content/drive/MyDrive/Colab Notebooks/sports/sports_competition/\343\203\206\343\202\231\343\203\274\343\202\277/mmaction_exe_uchimata_I_W_gameover_seianno/judo_uchimata_I_W/dlc-models/iteration-0/judo_uchimata_I_WDec21-trainset95shuffle1/train/snapshot-4000"
# all_model_checkpoint_paths: "/content/drive/MyDrive/Colab Notebooks/sports/sports_competition/\343\203\206\343\202\231\343\203\274\343\202\277/mmaction_exe_uchimata_I_W_gameover_seianno/judo_uchimata_I_W/dlc-models/iteration-0/judo_uchimata_I_WDec21-trainset95shuffle1/train/snapshot-1000"
# all_model_checkpoint_paths: "/content/drive/MyDrive/Colab Notebooks/sports/sports_competition/\343\203\206\343\202\231\343\203\274\343\202\277/mmaction_exe_uchimata_I_W_gameover_seianno/judo_uchimata_I_W/dlc-models/iteration-0/judo_uchimata_I_WDec21-trainset95shuffle1/train/snapshot-2000"
# all_model_checkpoint_paths: "/content/drive/MyDrive/Colab Notebooks/sports/sports_competition/\343\203\206\343\202\231\343\203\274\343\202\277/mmaction_exe_uchimata_I_W_gameover_seianno/judo_uchimata_I_W/dlc-models/iteration-0/judo_uchimata_I_WDec21-trainset95shuffle1/train/snapshot-3000"
# all_model_checkpoint_paths: "/content/drive/MyDrive/Colab Notebooks/sports/sports_competition/\343\203\206\343\202\231\343\203\274\343\202\277/mmaction_exe_uchimata_I_W_gameover_seianno/judo_uchimata_I_W/dlc-models/iteration-0/judo_uchimata_I_WDec21-trainset95shuffle1/train/snapshot-4000"

In [49]:
pose_cfg_path = '/content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/mmaction_exe_uchimata_I_W_gameover_seianno/judo_uchimata_I_W/dlc-models/iteration-0/judo_uchimata_I_WDec21-trainset95shuffle1/train/pose_cfg.yaml'

import webbrowser
webbrowser.open(pose_cfg_path)

print('Please edit init_weights to include the path to the last pre-trained model...')

Please edit init_weights to include the path to the last pre-trained model...


In [50]:
config_path = "/content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/mmaction_exe_uchimata_I_W_gameover_seianno/judo_uchimata_I_W/config.yaml"
deeplabcut.train_network(config_path, shuffle=1, displayiters=100, saveiters=1000, maxiters=50000, allow_growth=True)

Config:
{'Task': None,
 'TrainingFraction': None,
 'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16]],
 'all_joints_names': ['nose',
                      'left_eye',
                      'right_eye',
                      'left_ear',
                      'right_ear',
                      'left_shoulder',
                      'right_shoulder',
                      'left_elbow',
                      'right_elbow',
                      'left_wrist',
                      'right_wrist',
                      'left_hip',
                      'right_hip',
                      'left_knee',
                      'right_knee',
                      'left_ankle',
                

Selecting multi-animal trainer
Activating limb prediction...


FileNotFoundError: ignored

## Start evaluating:

 - First, we evaluate the pose estimation performance.

- This function evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images) and stores the results as .5 and .csv file in a subdirectory under **evaluation-results**

- If the scoremaps do not look accurate, don't proceed to tracklet assembly; please consider (1) adding more data, (2) adding more bodyparts!

- more info: https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html#evaluate-the-trained-network

Here is an example of what you'd aim to see before proceeding:

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1590535809087-X655WY9W1MW1MY1I7DHE/ke17ZwdGBToddI8pDm48kBoswZhKnUtAF7-bTXgw67EUqsxRUqqbr1mOJYKfIPR7LoDQ9mXPOjoJoqy81S2I8N_N4V1vUb5AoIIIbLZhVYxCRW4BPu10St3TBAUQYVKc5tTP1cnANTUwNNPnYFjIp6XbP9N1GxIgAkxvBVqt0UvLpPHYwvNQTwHg8f_Zu8ZF/evaluation.png?format=1000w)



In [ ]:
#let's evaluate first:
deeplabcut.evaluate_network(path_config_file,Shuffles=[shuffle], plotting=True)
#plot a few scoremaps:
deeplabcut.extract_save_all_maps(path_config_file, shuffle=shuffle, Indices=[0])

Running  DLC_dlcrnetms5_judo_wazaariDec11shuffle1_1000  with # of trainingiterations: 1000


/usr/local/lib/python3.10/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Activating extracting of PAFs
Network Evaluation underway...


30it [00:12,  2.43it/s]


Results for 1000 training iterations, training fraction of 95, and shuffle 1:
Train error: 6.06 pixels. Test error: 4.91 pixels.
With pcutoff of 0.6:
Train error: 5.31 pixels. Test error: 4.91 pixels.
##########################################
Average Euclidean distance to GT per individual (in pixels; test-only)
individuals
individual1    5.718041
individual2    4.724917
Average Euclidean distance to GT per bodypart (in pixels; test-only)
bodyparts
left_ankle        4.253663
left_ear          4.720641
left_elbow        5.576006
left_eye          6.412545
left_hip          4.868287
left_knee         5.191362
left_shoulder     6.055222
left_wrist        3.546989
nose              6.277965
right_ankle       2.672548
right_ear         3.086850
right_elbow       4.888761
right_eye         5.611404
right_hip         3.549226
right_knee        6.141751
right_shoulder    5.106392
right_wrist       5.912508
Done and results stored for snapshot:  snapshot-1000
Graph 1|1


100%|██████████| 30/30 [00:00<00:00, 1888.33it/s]
/usr/local/lib/python3.10/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Activating extracting of PAFs
Analyzing data...


1it [00:00,  1.83it/s]


Saving plots...


100%|██████████| 1/1 [00:05<00:00,  5.67s/it]


IF these images, numbers, and maps do not look good, do not proceed. You should increase the diversity and number of frames you label, and re-create a training dataset and re-train!

## Start Analyzing videos:
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in a pickle file in the same directory where the video resides.

In [ ]:
print("Start Analyzing my video(s)!")
#EDIT OPTION: which video(s) do you want to analyze? You can pass a path or a folder:
# currently, if you run "as is" it assumes you have a video in the DLC project video folder!

deeplabcut.analyze_videos(path_config_file,videofile_path, shuffle=shuffle, videotype=VideoType)

Start Analyzing my video(s)!
Using snapshot-1000 for model /content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/mmaction_exe_uchimata_I_W_gameover_seianno/judo_wazaari/dlc-models/iteration-0/judo_wazaariDec11-trainset95shuffle1
Activating extracting of PAFs
Analyzing all the videos in the directory...
Starting to analyze %  /content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/mmaction_exe_uchimata_I_W_gameover_seianno/judo_wazaari/videos/2019WCTBM.mp4
Loading  /content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/mmaction_exe_uchimata_I_W_gameover_seianno/judo_wazaari/videos/2019WCTBM.mp4
Duration of video [s]:  10.0 , recorded with  1.0 fps!
Overall # of frames:  10  found with (before cropping) frame dimensions:  640 368
Starting to extract posture from the video(s) with batchsize: 8


100%|██████████| 10/10 [00:04<00:00,  2.30it/s]

Video Analyzed. Saving results in /content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/mmaction_exe_uchimata_I_W_gameover_seianno/judo_wazaari/videos...
Using snapshot-1000 for model /content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/mmaction_exe_uchimata_I_W_gameover_seianno/judo_wazaari/dlc-models/iteration-0/judo_wazaariDec11-trainset95shuffle1
Processing...  /content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/mmaction_exe_uchimata_I_W_gameover_seianno/judo_wazaari/videos/2019WCTBM.mp4
Analyzing /content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/mmaction_exe_uchimata_I_W_gameover_seianno/judo_wazaari/videos/2019WCTBMDLC_dlcrnetms5_judo_wazaariDec11shuffle1_1000.h5



100%|██████████| 10/10 [00:00<00:00, 527.94it/s]
10it [00:00, 1505.60it/s]


The tracklets were created (i.e., under the hood deeplabcut.convert_detections2tracklets was run). Now you can 'refine_tracklets' in the GUI, or run 'deeplabcut.stitch_tracklets'.
Processing...  /content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/mmaction_exe_uchimata_I_W_gameover_seianno/judo_wazaari/videos/2019WCTBM.mp4


...

100%|██████████| 10/10 [00:04<00:00,  2.45it/s]

Video Analyzed. Saving results in /content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/mmaction_exe_uchimata_I_W_gameover_seianno/judo_wazaari/videos...
Using snapshot-1000 for model /content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/mmaction_exe_uchimata_I_W_gameover_seianno/judo_wazaari/dlc-models/iteration-0/judo_wazaariDec11-trainset95shuffle1
Processing...  /content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/mmaction_exe_uchimata_I_W_gameover_seianno/judo_wazaari/videos/2019WCTSP.mp4
Analyzing /content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/mmaction_exe_uchimata_I_W_gameover_seianno/judo_wazaari/videos/2019WCTSPDLC_dlcrnetms5_judo_wazaariDec11shuffle1_1000.h5



100%|██████████| 10/10 [00:00<00:00, 425.61it/s]
10it [00:00, 1431.99it/s]


The tracklets were created (i.e., under the hood deeplabcut.convert_detections2tracklets was run). Now you can 'refine_tracklets' in the GUI, or run 'deeplabcut.stitch_tracklets'.
Processing...  /content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/mmaction_exe_uchimata_I_W_gameover_seianno/judo_wazaari/videos/2019WCTSP.mp4


OSError: ignored

Optional: Now you have the option to check the raw dections before animals are assembled. To do so, pass a video path:

In [ ]:
# # !cp -r "/content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/3second_videos/uchimata_I/back/2018WCB_100_P1_WOLF Aaron_JPN_HAJAS Matej_SVK_1.mp4" "/content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/judo_wazaari-sei-2023-12-11/test_video"
# !cp -r "/content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/3second_videos/uchimata_I/right/2018WCB_52_F_ABE Uta_JPN_SHISHIME Ai_JPN_1.mp4" "/content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/judo_wazaari-sei-2023-12-11/test_video"

In [ ]:
# ##### PROTIP: #####
# ## look at the output video; if the pose estimation (i.e. key points)
# ## don't look good, don't proceed with tracking - add more data to your training set and re-train!

# #EDIT: let's check a specific video (PLEASE EDIT VIDEO PATH):
# Specific_videofile = '/content/drive/MyDrive/DeepLabCut_maDLC_DemoData/MontBlanc-Daniel-2019-12-16/videos/short.mov'

# #don't edit:
# deeplabcut.create_video_with_all_detections(path_config_file, [Specific_videofile], shuffle=shuffle)

If the resutling video (ends in full.mp4) is not good, we highly recommend adding more data and training again. See here: https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html#decision-break-point

# Next, we will assemble animals using our data-driven optimal graph method:

- Here, we will find the optimal graph, which matches the "data-driven" method from our paper (Figure adapted from Lauer et al. 2021):

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1626266017809-XO6NX84QB4FBAZGOTCEY/fig3.jpg?format=400w)


- note, you can set the number of animals you expect to see, so check, edit, then click run:

In [ ]:
videofile_path = '/content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/judo_wazaari-sei-2023-12-11/test_video'

In [ ]:
# path_config_file = "/content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/mmaction_exe_uchimata_I_W_gameover_seianno/judo_ippon/config.yaml"

In [ ]:
#Check and edit:
numAnimals = 2 #how many animals do you expect to find?
tracktype= 'ellipse' #box, skeleton, ellipse:
#-- ellipse is recommended, unless you have a single-point ma project, then use BOX!

#Optional:
#imagine you tracked a point that is not useful for assembly,
#like a tail tip that is far from the body, consider dropping it for this step (it's still used later)!
#To drop it, uncomment the next line TWO lines and add your parts(s):

#bodypart= 'Tail_end'
#deeplabcut.convert_detections2tracklets(path_config_file, videofile_path, videotype=VideoType, shuffle=shuffle, overwrite=True, ignore_bodyparts=[bodypart])

#OR don't drop, just click RUN:
deeplabcut.convert_detections2tracklets(path_config_file, videofile_path, videotype=VideoType,
                                        shuffle=shuffle, overwrite=True, )

deeplabcut.stitch_tracklets(path_config_file, videofile_path, shuffle=shuffle, track_method=tracktype, n_tracks=numAnimals)

Now let's filter the data to remove any small jitter:

In [ ]:
deeplabcut.filterpredictions(path_config_file,
                                 videofile_path,
                                 shuffle=shuffle,
                                 videotype=VideoType,
                                 track_method = tracktype, )

Analyzing all the videos in the directory...
Filtering with median model /content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/judo_wazaari-sei-2023-12-11/test_video/2018WCB_52_F_ABE Uta_JPN_SHISHIME Ai_JPN_1.mp4
Saving filtered csv poses!
Filtering with median model /content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/judo_wazaari-sei-2023-12-11/test_video/2018WCB_100_P1_WOLF Aaron_JPN_HAJAS Matej_SVK_1.mp4
Saving filtered csv poses!
Filtering with median model /content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/judo_wazaari-sei-2023-12-11/test_video/2018WCB_60_3M_LEE Harim_KOR_NAGAYAMA Ryuju_JPN_1.mp4
Saving filtered csv poses!


## Create plots of your trajectories:

In [ ]:
deeplabcut.plot_trajectories(path_config_file, videofile_path, videotype=VideoType, shuffle=shuffle, track_method=tracktype)

Analyzing all the videos in the directory...
Loading  /content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/judo_wazaari-sei-2023-12-11/test_video/2018WCB_60_3M_LEE Harim_KOR_NAGAYAMA Ryuju_JPN_1.mp4 and data.
Loading  /content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/judo_wazaari-sei-2023-12-11/test_video/2018WCB_52_F_ABE Uta_JPN_SHISHIME Ai_JPN_1.mp4 and data.
Loading  /content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/judo_wazaari-sei-2023-12-11/test_video/2018WCB_100_P1_WOLF Aaron_JPN_HAJAS Matej_SVK_1.mp4 and data.
Plots created! Please check the directory "plot-poses" within the video directory


Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4.

## Create labeled video:
This function is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides.

In [ ]:
deeplabcut.create_labeled_video(path_config_file,
                                videofile_path,
                                shuffle=shuffle,
                                color_by="bodypart",
                                videotype=VideoType,
                                save_frames=False,
                                filtered=True, draw_skeleton=True, )

Analyzing all the videos in the directory...
Starting to process video: /content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/judo_wazaari-sei-2023-12-11/test_video/2018WCB_60_3M_LEE Harim_KOR_NAGAYAMA Ryuju_JPN_1.mp4Starting to process video: /content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/judo_wazaari-sei-2023-12-11/test_video/2018WCB_52_F_ABE Uta_JPN_SHISHIME Ai_JPN_1.mp4Starting to process video: /content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/judo_wazaari-sei-2023-12-11/test_video/2018WCB_100_P1_WOLF Aaron_JPN_HAJAS Matej_SVK_1.mp4


Loading /content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/judo_wazaari-sei-2023-12-11/test_video/2018WCB_52_F_ABE Uta_JPN_SHISHIME Ai_JPN_1.mp4 and data.Loading /content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/judo_wazaari-sei-2023-12-11/test_video/2018WCB_100_P1_WOLF Aaron_JPN_HAJAS Matej_SVK_1.mp4 and data.Loading /content/drive/MyDrive/Colab Notebooks/2023年度スポコン/データ/judo_wazaari-sei-2023-12-11/test_video/2018WCB_60_

  0%|          | 0/73 [00:00<?, ?it/s]


Generating frames and creating video.

Generating frames and creating video.


100%|██████████| 73/73 [00:00<00:00, 116.65it/s]


[True, True, True]